## Transformações e Ações de objetos RDD

In [ ]:
%%writefile example2.txt
first
second line
the third line
then a fourth line

In [ ]:
from pyspark import SparkContext

In [ ]:
sc = SparkContext()

In [ ]:
sc.textFile('example2.txt')

In [ ]:
text_rdd = sc.textFile('example2.txt')

In [ ]:
text_rdd.map(lambda line: line.split())

In [ ]:
words = text_rdd.map(lambda line: line.split())

In [ ]:
words.collect()

In [ ]:
text_rdd.collect()

In [ ]:
text_rdd.flatMap(lambda line: line.split()).collect()

In [ ]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00
207       10/20/2017      204       TX       173          1000.00
208       10/21/2017      204       TX       173          1500.00

In [ ]:
services = sc.textFile('services.txt')

In [ ]:
services.take(2)

In [ ]:
services.map(lambda line: line.split()).take(3)

### Remover # do header

In [ ]:
services.map(lambda line: line[1:] if line[0] == '#' else line).collect()

In [ ]:
clean = services.map(lambda line: line[1:] if line[0] == '#' else line)

In [ ]:
clean = clean.map(lambda line: line.split())

In [ ]:
clean.collect()

### Obtendo dados

In [ ]:
pairs = clean.map(lambda lst: (lst[3], lst[-1]))

In [ ]:
rekey = pairs.reduceByKey(lambda amt1, amt2: amt1+amt2)

In [ ]:
rekey.collect()

In [ ]:
rekey = pairs.reduceByKey(lambda amt1, amt2: float(amt1)+float(amt2))

In [ ]:
rekey.collect()

### Transformação completa dos dados

In [ ]:
from operator import add

In [ ]:
# Pegar os dados (State, Amount)
step1 = clean.map(lambda lst: (lst[3], lst[-1]))

# Transformar amount em float
step2 = step1.map(lambda row: (row[0], float(row[1])) if not 'State' in row[0] else (row[0], row[1]))

# Reduce by Key
#step2 = step1.reduceByKey(lambda amt1, amt2: [float(amt1), float(amt2)])
step3 = step2.reduceByKey(add)

# Retirar os nomes das colunas
#step3 = step2.filter(lambda x: not x[0]=="State")

# Ordenar os resultados por Amount
#step4 = step3.sortBy(lambda stAmount: stAmount[1], ascending=False)

# Ação
step3.collect()


In [ ]:
### Unpacking
step1.map(lambda row: (row[0], float(row[1])) if not 'State' in row[0] else (row[0], row[1])).collect()